In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split,current_timestamp,timestamp_seconds,window
from pyspark.sql.types import TimestampType


sc = SparkContext()

sc.setLogLevel('ERROR')


spark = SparkSession \
    .builder \
    .appName('ContarPalavrasEstruturado') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 13:10:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/22 13:10:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
linhas = spark.readStream \
    .format("socket") \
    .option("host","localhost") \
    .option("port",3456)\
    .option("includeTimestamp","true")\
    .load()

In [3]:
palavras = linhas.select(
            explode(
                split(linhas.value, " ")
            ).alias("palavra"),
            linhas.timestamp
)

In [4]:
contagem = palavras.groupBy(

        window(palavras.timestamp, "10 minutes", "5 minutes"),
        palavras.palavra).count().orderBy('window')

In [6]:
consulta = contagem.writeStream\
        .outputMode("complete")\
        .format("console")\
        .option("truncate","false")\
        .start()


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|palavra|count|
+------+-------+-----+
+------+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |palavra|count|
+------------------------------------------+-------+-----+
|{2022-11-22 13:05:00, 2022-11-22 13:15:00}|ver    |3    |
|{2022-11-22 13:10:00, 2022-11-22 13:20:00}|ver    |3    |
+------------------------------------------+-------+-----+

